In [17]:
%pip install pyvis



[notice] A new release of pip is available: 25.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [18]:
import pandas as pd
import networkx as nx
from pyvis.network import Network
from pathlib import Path
import json


In [19]:
# ============ Configuration ============
INPUT_CSV = "hp-dialogues.csv"

# ============ Data Loading ============
print("📊 Loading dialogue data...")
df = pd.read_csv(INPUT_CSV)
print(f"✅ Loaded {len(df)} dialogue entries")
print(f"📝 Unique speakers: {df['speaker'].nunique()}")
print(f"🎯 Unique targets: {df['target'].nunique()}")
df.head()


📊 Loading dialogue data...
✅ Loaded 24528 dialogue entries
📝 Unique speakers: 190
🎯 Unique targets: 311


,book,speaker,target,sentiment
0,1,Vernon Dursley,Vernon Dursley,positive
1,1,Vernon Dursley,Petunia Dursley,positive
2,1,Vernon Dursley,Dudley Dursley,positive
3,1,Petunia Dursley,Lily Potter,negative
4,1,Petunia Dursley,James Potter,negative


In [20]:
# ============ Data Aggregation ============
print("🔄 Aggregating dialogue data by speaker-target pairs...")

# Collapse edges (speaker → target): total weight + dominant sentiment
agg = df.groupby(["speaker", "target"]).agg(
    weight=("sentiment", "size"),
    pos_count=("sentiment", lambda x: (x == "positive").sum()),
    neg_count=("sentiment", lambda x: (x == "negative").sum()),
    neu_count=("sentiment", lambda x: (x == "neutral").sum()),
).reset_index()

def pick_sentiment(row):
    """Determine dominant sentiment for a relationship"""
    counts = {"positive": row["pos_count"], "negative": row["neg_count"], "neutral": row["neu_count"]}
    mx = max(counts.values())
    winners = [s for s, c in counts.items() if c == mx]
    # Priority: negative > positive > neutral (in case of ties)
    if "negative" in winners: return "negative"
    if "positive" in winners: return "positive"
    return "neutral"

agg["sentiment"] = agg.apply(pick_sentiment, axis=1)

print(f"✅ Created {len(agg)} unique speaker-target relationships")
agg.head(10)


🔄 Aggregating dialogue data by speaker-target pairs...
✅ Created 3493 unique speaker-target relationships


,speaker,target,weight,pos_count,neg_count,neu_count,sentiment
0,Aberforth Dumbledore,Albus Dumbledore,11,0,10,1,negative
1,Aberforth Dumbledore,Ariana Dumbledore,3,0,3,0,negative
2,Aberforth Dumbledore,Death Eater (Dolohov),1,0,0,1,neutral
3,Aberforth Dumbledore,Death Eaters,6,0,5,1,negative
4,Aberforth Dumbledore,Dementors,1,0,1,0,negative
5,Aberforth Dumbledore,Dobby,2,2,0,0,positive
6,Aberforth Dumbledore,Elphias Doge,2,0,1,1,negative
7,Aberforth Dumbledore,Gellert Grindelwald,4,0,4,0,negative
8,Aberforth Dumbledore,Giants,1,0,0,1,neutral
9,Aberforth Dumbledore,Harry Potter,17,2,7,8,neutral


In [21]:
# ============ Graph Setup ============
print("🕸️ Building network graph and calculating layout...")

# Build graph (for layout)
G = nx.from_pandas_edgelist(
    agg, source="speaker", target="target", edge_attr=["sentiment", "weight"], create_using=nx.DiGraph()
)

# Calculate positions using spring layout
pos = nx.spring_layout(G, weight="weight", k=5.0, iterations=150, seed=42)

print(f"✅ Graph created with {len(G.nodes())} nodes and {len(G.edges())} edges")


🕸️ Building network graph and calculating layout...
✅ Graph created with 315 nodes and 3493 edges


In [22]:
# ============ Graph Data Builder Function ============
def remarks_to_color(neg, pos, neu):
    """Convert sentiment counts to RGB color using weighted color mixing."""
    total = pos + neg + neu
    if total == 0:
        return "rgb(128,128,128)"  # Default gray
    
    # Normalize to percentages
    pos_pct = pos / total
    neg_pct = neg / total
    neu_pct = neu / total
    
    # Define base colors
    colors = {
        'positive': {'r': 34, 'g': 197, 'b': 94},   # Green
        'negative': {'r': 239, 'g': 68, 'b': 68},   # Red
        'neutral': {'r': 107, 'g': 114, 'b': 128}   # Gray
    }
    
    # Calculate weighted color mixing
    r = round(
        colors['positive']['r'] * pos_pct + 
        colors['negative']['r'] * neg_pct + 
        colors['neutral']['r'] * neu_pct
    )
    g = round(
        colors['positive']['g'] * pos_pct + 
        colors['negative']['g'] * neg_pct + 
        colors['neutral']['g'] * neu_pct
    )
    b = round(
        colors['positive']['b'] * pos_pct + 
        colors['negative']['b'] * neg_pct + 
        colors['neutral']['b'] * neu_pct
    )
    
    return f"rgb({r},{g},{b})"

def build_graph_data(mode: str):
    """Return nodes + edges for a given mode (speaker/target)."""
    if mode == "speaker":
        counts = df["speaker"].value_counts().to_dict()
        def comments_of(char): return df[df["speaker"] == char]
        title_prefix = "remarks made"
    else:
        counts = df["target"].value_counts().to_dict()
        def comments_of(char): return df[df["target"] == char]
        title_prefix = "remarks received"

    all_chars = set(df["speaker"].unique()) | set(df["target"].unique())
    node_colors = {}
    for char in all_chars:
        cc = comments_of(char)
        pos_c = (cc["sentiment"] == "positive").sum()
        neg_c = (cc["sentiment"] == "negative").sum()
        neu_c = (cc["sentiment"] == "neutral").sum()
        
        node_colors[char] = remarks_to_color(neg_c, pos_c, neu_c)

    max_count = max(counts.values()) if counts else 1
    min_size, max_size = 10, 50

    nodes = []
    for node in G.nodes():
        ccount = counts.get(node, 0)
        size = min_size + (max_size - min_size) * (ccount / max_count)
        color = node_colors.get(node, "rgb(128,128,128)")
        x, y = pos[node]
        
        # Calculate sentiment percentages for tooltip
        cc = comments_of(node)
        pos_c = (cc["sentiment"] == "positive").sum()
        neg_c = (cc["sentiment"] == "negative").sum()
        neu_c = (cc["sentiment"] == "neutral").sum()
        total_sentiment = pos_c + neg_c + neu_c
        
        if total_sentiment > 0:
            pos_pct = (pos_c / total_sentiment) * 100
            neg_pct = (neg_c / total_sentiment) * 100
            neu_pct = (neu_c / total_sentiment) * 100
            sentiment_info = f"{pos_pct:.1f}% pos, {neg_pct:.1f}% neg, {neu_pct:.1f}% neu"
        else:
            sentiment_info = "no sentiment data"
        
        nodes.append({
            "id": node,
            "label": node,
            "title": f"{node}: {ccount} {title_prefix} ({sentiment_info})",
            "color": color,
            "size": size,
            "x": float(x * 5500),
            "y": float(y * 5500),
            "physics": False,
            "shape": "dot"
        })

    edges = []
    mx_w = agg["weight"].max()
    for _, row in agg.iterrows():
        u, v = row["speaker"], row["target"]
        sentiment, w = row["sentiment"], row["weight"]
        width = 1 + 14 * (w / mx_w)
        
        # Calculate edge color using the new red-green-yellow formula
        pos_c = row["pos_count"]
        neg_c = row["neg_count"]
        neu_c = row["neu_count"]
        
        edge_color = remarks_to_color(neg_c, pos_c, neu_c)
        
        # Calculate percentages for tooltip
        total = pos_c + neg_c + neu_c
        if total > 0:
            pos_pct = (pos_c / total) * 100
            neg_pct = (neg_c / total) * 100
            neu_pct = (neu_c / total) * 100
            sentiment_info = f"{pos_pct:.1f}% pos, {neg_pct:.1f}% neg, {neu_pct:.1f}% neu"
        else:
            sentiment_info = "no sentiment data"
        
        edges.append({
            "from": u,
            "to": v,
            "color": edge_color,
            "width": width,
            "title": f"{w} remarks ({sentiment}) - {sentiment_info}",
            "arrows": {"to": {"enabled": True, "scaleFactor": 0.5, "type": "arrow"}}
        })

    return {"nodes": nodes, "edges": edges}

print("✅ Graph data builder function defined")


✅ Graph data builder function defined


In [23]:
# ============ Generate Graph Data ============
print("📊 Generating data for both viewing modes...")

speaker_data = build_graph_data("speaker")
target_data = build_graph_data("target")

print(f"✅ Speaker mode: {len(speaker_data['nodes'])} nodes, {len(speaker_data['edges'])} edges")
print(f"✅ Target mode: {len(target_data['nodes'])} nodes, {len(target_data['edges'])} edges")


📊 Generating data for both viewing modes...
✅ Speaker mode: 315 nodes, 3493 edges
✅ Target mode: 315 nodes, 3493 edges


In [24]:
# ============ Generate HTML Visualization ============
print("🌐 Creating interactive HTML visualization...")

html_template = f"""
<html>
<head>
<meta charset="utf-8"/>
<script type="text/javascript" src="https://unpkg.com/vis-network/standalone/umd/vis-network.min.js"></script>
<style>
* {{
  margin: 0;
  padding: 0;
  box-sizing: border-box;
}}
html, body {{
  height: 100%;
  overflow: hidden;
  font-family: sans-serif;
}}
.controls {{
  position: fixed;
  top: 10px;
  left: 10px;
  z-index: 1000;
  background: rgba(255,255,255,0.95);
  padding: 8px;
  border-radius: 8px;
  box-shadow: 0 2px 8px rgba(0,0,0,0.15);
}}
button {{
  margin-right: 8px;
  padding: 6px 12px;
  border-radius: 6px;
  border: 1px solid #ccc;
  background: #f7f7f7;
  cursor: pointer;
  font-size: 14px;
}}
button:hover {{
  background: #e8f5e9;
}}
button.active {{
  background: #e8f5e9;
  border-color: #a5d6a7;
  font-weight: bold;
}}
#network {{
  width: 100vw;
  height: 100vh;
  border: none;
}}
#legend-bubble {{
  position: fixed;
  right: 12px;
  bottom: 12px;
  background: rgba(255,255,255,0.95);
  border: 1px solid #ccc;
  border-radius: 10px;
  padding: 10px 14px;
  font: 13px/1.4 sans-serif;
  box-shadow: 0 2px 8px rgba(0,0,0,0.2);
  max-width: 300px;
  z-index: 1000;
}}
</style>
</head>
<body>
<div class="controls">
  <button id="btnSpeaker" class="active" onclick="showGraph('speaker')">Speaker Graph</button>
  <button id="btnTarget" onclick="showGraph('target')">Target Graph</button>
  <button id="btnEdges" onclick="toggleEdges()">Hide Edges</button>
</div>
<div id="network"></div>

<div id="legend-bubble"></div>

<script>
var speakerData = {json.dumps(speaker_data)};
var targetData = {json.dumps(target_data)};

var legendSpeaker = `
  <strong>Graph Legend – Speaker Mode</strong><br>
  • Node size = # remarks <b>made</b><br>
  • Node color = sentiment mix of <b>remarks made</b><br>
  • Edge size = # remarks <b>made</b> about the targeted character<br>
  • Edge color = sentiment mix of <b>remarks made</b> about the targeted character<br><br>
  <strong>How to Use</strong><br>
  • Click node = show only its outgoing edges<br>
  • Double-click background = reset<br>
  • "Hide/Show Edges" = toggle all edges
`;

var legendTarget = `
  <strong>Graph Legend – Target Mode</strong><br>
  • Node size = # remarks <b>received</b><br>
  • Node color = sentiment mix of <b>received remarks</b><br>
  • Edge size = # remarks <b>received</b> by the character<br>
  • Edge color = sentiment mix of <b>received remarks</b> by the character<br><br>
  <strong>How to Use</strong><br>
  • Click node = show only its incoming edges<br>
  • Double-click background = reset<br>
  • "Hide/Show Edges" = toggle all edges
`;

var current = 'speaker';
var edgesHidden = false;
var container = document.getElementById('network');
var network, nodes, edges, originalEdgesCopy;

function drawGraph(data){{
  nodes = new vis.DataSet(data.nodes);
  edges = new vis.DataSet(data.edges);
  originalEdgesCopy = new vis.DataSet(data.edges);

  var options = {{
    interaction: {{ hover: true }},
    physics: false,
    nodes: {{ font: {{ size: 14 }} }}
  }};
  network = new vis.Network(container, {{nodes: nodes, edges: edges}}, options);

  // Click filtering
  network.on("selectNode", function(params){{
    if(params.nodes.length){{
      var nodeId = params.nodes[0];
      var keepEdges;
      if(current === 'speaker'){{
        keepEdges = originalEdgesCopy.get({{filter: function(e){{return e.from===nodeId;}}}});
      }} else {{
        keepEdges = originalEdgesCopy.get({{filter: function(e){{return e.to===nodeId;}}}});
      }}
      var nbrs = new Set([nodeId]);
      keepEdges.forEach(function(e){{nbrs.add(e.from); nbrs.add(e.to);}});
      var updates = nodes.get().map(function(n){{return {{id:n.id, hidden:!nbrs.has(n.id)}};}});
      edges.clear(); edges.add(keepEdges); nodes.update(updates);
    }}
  }});
  network.on("doubleClick", function(params){{ resetAll(); }});
}}

function resetAll(){{
  nodes.update(nodes.get().map(function(n){{return {{id:n.id, hidden:false}};}}));
  edges.clear(); edges.add(originalEdgesCopy.get());
}}

function showGraph(which){{
  current = which;
  document.getElementById('btnSpeaker').classList.remove('active');
  document.getElementById('btnTarget').classList.remove('active');
  document.getElementById('btn'+which.charAt(0).toUpperCase()+which.slice(1)).classList.add('active');
  edgesHidden = false;
  document.getElementById("btnEdges").innerText = "Hide Edges";
  if(which==='speaker') {{
    drawGraph(speakerData);
    document.getElementById("legend-bubble").innerHTML = legendSpeaker;
  }} else {{
    drawGraph(targetData);
    document.getElementById("legend-bubble").innerHTML = legendTarget;
  }}
}}

function toggleEdges(){{
  if(!edgesHidden){{
    edges.clear();
    document.getElementById("btnEdges").innerText = "Show Edges";
    edgesHidden = true;
  }} else {{
    edges.clear(); edges.add(originalEdgesCopy.get());
    document.getElementById("btnEdges").innerText = "Hide Edges";
    edgesHidden = false;
  }}
}}

showGraph('speaker');
</script>
</body>
</html>
"""

# Write the HTML file
output_file = "hp_conversation_graph_toggle.html"
Path(output_file).write_text(html_template, encoding="utf-8")

print(f"✅ Interactive visualization saved as '{output_file}'")
print("🌐 Open the HTML file in your browser to explore the character relationships!")
print("📖 Use the buttons to switch between Speaker and Target modes")
print("🖱️  Click nodes to filter connections, double-click background to reset")


🌐 Creating interactive HTML visualization...
✅ Interactive visualization saved as 'hp_conversation_graph_toggle.html'
🌐 Open the HTML file in your browser to explore the character relationships!
📖 Use the buttons to switch between Speaker and Target modes
🖱️  Click nodes to filter connections, double-click background to reset
